## Monitoramento de Queimadas no Brasil entre os anos de 2003 a 2024

###  ADENILSON SILVA

## EXTRAÇÃO, TRANSFORMAÇÃO E CARRAGAMENTO DE DADOS

###  1 - Importando bibliotecas e criando funções

In [1]:
import pandas as pd  # Manipulação de dados tabulares (DataFrame)
import numpy as np  # Operações com arrays e funções matemáticas
import matplotlib.pyplot as plt  # Criação de gráficos
import basedosdados as bd  # Acesso a dados públicos da Base dos Dados
import geopandas as gpd  # Análise e manipulação de dados geoespaciais
import folium  # Criação de mapas interativos
from folium.features import GeoJsonTooltip  # Tooltips interativos em mapas
import random  # Geração de números aleatórios
from shapely.geometry import Point  # Representação de pontos geográficos
import calendar  # Manipulação de datas, obtenção de nomes de meses, dias etc.
import locale  # Configuração de localidade (para formatar datas, moedas etc.)

locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

In [2]:
def estacao_ano(mes):
    if mes in [12, 1, 2]:
        return 'Verão'
    elif mes in [3, 4, 5]:
        return 'Outono'
    elif mes in [6, 7, 8]:
        return 'Inverno'
    else:
        return 'Primavera'

def classificar_risco(valor):
    if valor <= .15:
        return 'Mínimo'
    elif valor <= .40:
        return 'Baixo'
    elif valor <= .70:
        return 'Médio'
    elif valor <= .95:
        return 'Alto'
    elif valor > .95 :
        return 'Crítico'
    else:
        return 'Indeterminado'

### 2 - Leitura dos dados

In [3]:
billing_id = "uso-de-dados-gis-vetoriais"

query = """
  SELECT
    dados.ano as ano,
    dados.mes as mes,
    dados.data_hora as data_hora,
    dados.bioma as bioma,
    dados.sigla_uf AS sigla_uf,
    diretorio_sigla_uf.nome AS sigla_uf_nome,
    dados.id_municipio AS id_municipio,
    diretorio_id_municipio.nome AS id_municipio_nome,
    dados.latitude as latitude,
    dados.longitude as longitude,
    dados.satelite as satelite,
    dados.dias_sem_chuva as dias_sem_chuva,
    dados.precipitacao as precipitacao,
    dados.risco_fogo as risco_fogo,
    dados.potencia_radiativa_fogo as potencia_radiativa_fogo
FROM `basedosdados.br_inpe_queimadas.microdados` AS dados
LEFT JOIN (SELECT DISTINCT sigla,nome  FROM `basedosdados.br_bd_diretorios_brasil.uf`) AS diretorio_sigla_uf
    ON dados.sigla_uf = diretorio_sigla_uf.sigla
LEFT JOIN (SELECT DISTINCT id_municipio,nome  FROM `basedosdados.br_bd_diretorios_brasil.municipio`) AS diretorio_id_municipio
    ON dados.id_municipio = diretorio_id_municipio.id_municipio
    --WHERE dados.ano IN (2024) and dados.bioma = 'Amazônia'
"""

#data = bd.read_sql(query = query, billing_project_id = billing_id)
#data.to_parquet('dados_brutos.parquet')

In [4]:
df = pd.read_parquet('dados_brutos.parquet')
df.head()

,ano,mes,data_hora,bioma,sigla_uf,sigla_uf_nome,id_municipio,id_municipio_nome,latitude,longitude,satelite,dias_sem_chuva,precipitacao,risco_fogo,potencia_radiativa_fogo
0,2024,7,2024-07-27 12:56:20,Cerrado,BA,Bahia,2909307,Correntina,-13.16580,-45.22000,GOES-16,107.0,0.0,1.0,125.8
1,2024,7,2024-07-27 15:26:20,Cerrado,BA,Bahia,2909307,Correntina,-13.16580,-45.22000,GOES-16,107.0,0.0,1.0,77.1
2,2024,7,2024-07-27 00:16:20,Cerrado,BA,Bahia,2909307,Correntina,-13.16580,-45.22000,GOES-16,107.0,0.0,1.0,206.2
3,2024,7,2024-07-27 00:56:20,Cerrado,BA,Bahia,2909307,Correntina,-13.18510,-45.21680,GOES-16,107.0,0.0,1.0,163.7
4,2024,7,2024-07-29 17:28:00,Cerrado,BA,Bahia,2917334,Iuiú,-14.43488,-43.43603,AQUA_M-T,107.0,0.0,1.0,350.8


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17542900 entries, 0 to 17542899
Data columns (total 15 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   ano                      Int64         
 1   mes                      Int64         
 2   data_hora                datetime64[us]
 3   bioma                    object        
 4   sigla_uf                 object        
 5   sigla_uf_nome            object        
 6   id_municipio             object        
 7   id_municipio_nome        object        
 8   latitude                 float64       
 9   longitude                float64       
 10  satelite                 object        
 11  dias_sem_chuva           float64       
 12  precipitacao             float64       
 13  risco_fogo               float64       
 14  potencia_radiativa_fogo  float64       
dtypes: Int64(2), datetime64[us](1), float64(6), object(6)
memory usage: 2.0+ GB


In [6]:
df.describe()

,ano,mes,data_hora,latitude,longitude,dias_sem_chuva,precipitacao,risco_fogo,potencia_radiativa_fogo
count,17542900.0,17542900.0,17542900,1.754290e+07,1.754290e+07,1.261390e+07,1.261390e+07,1.261390e+07,1.242580e+07
mean,2020.353485,8.586238,2021-01-10 02:37:35.872197,-9.810282e+00,-5.248358e+01,1.914310e+01,6.733715e-01,-9.103549e+00,3.901071e+01
min,2003.0,1.0,2003-01-01 16:04:00,-3.374046e+01,-7.386338e+01,-9.990000e+02,0.000000e+00,-9.990000e+02,-3.800000e+00
25%,2021.0,8.0,2021-09-22 17:54:00,-1.317000e+01,-5.740360e+01,4.000000e+00,0.000000e+00,6.100000e-01,3.800000e+00
50%,2023.0,9.0,2023-11-15 18:17:00,-9.109700e+00,-5.206842e+01,1.100000e+01,0.000000e+00,1.000000e+00,1.030000e+01
75%,2024.0,10.0,2024-09-01 17:28:00,-5.984810e+00,-4.672232e+01,4.000000e+01,0.000000e+00,1.000000e+00,5.830000e+01
max,2025.0,12.0,2025-02-12 23:10:00,5.217610e+00,-3.479613e+01,1.200000e+02,2.529800e+02,1.000000e+00,8.094300e+03
std,6.306526,2.115722,NaN,5.981973e+00,7.287649e+00,1.076456e+02,3.401407e+00,9.891727e+01,7.127106e+01


### 3 - Tratamento de dados inconsistentes

In [7]:
df['risco_fogo'] = np.where(df.risco_fogo < 0, 0, df.risco_fogo)
df['dias_sem_chuva'] = np.where(df.dias_sem_chuva < 0, 0, df.dias_sem_chuva)
df['potencia_radiativa_fogo'] = np.where(df.potencia_radiativa_fogo < 0, 0, df.potencia_radiativa_fogo)

In [8]:
df.describe()

,ano,mes,data_hora,latitude,longitude,dias_sem_chuva,precipitacao,risco_fogo,potencia_radiativa_fogo
count,17542900.0,17542900.0,17542900,1.754290e+07,1.754290e+07,1.261390e+07,1.261390e+07,1.261390e+07,1.242580e+07
mean,2020.353485,8.586238,2021-01-10 02:37:35.872197,-9.810282e+00,-5.248358e+01,2.897320e+01,6.733715e-01,7.731182e-01,3.901071e+01
min,2003.0,1.0,2003-01-01 16:04:00,-3.374046e+01,-7.386338e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2021.0,8.0,2021-09-22 17:54:00,-1.317000e+01,-5.740360e+01,4.000000e+00,0.000000e+00,6.100000e-01,3.800000e+00
50%,2023.0,9.0,2023-11-15 18:17:00,-9.109700e+00,-5.206842e+01,1.100000e+01,0.000000e+00,1.000000e+00,1.030000e+01
75%,2024.0,10.0,2024-09-01 17:28:00,-5.984810e+00,-4.672232e+01,4.000000e+01,0.000000e+00,1.000000e+00,5.830000e+01
max,2025.0,12.0,2025-02-12 23:10:00,5.217610e+00,-3.479613e+01,1.200000e+02,2.529800e+02,1.000000e+00,8.094300e+03
std,6.306526,2.115722,NaN,5.981973e+00,7.287649e+00,3.597677e+01,3.401407e+00,3.345552e-01,7.127105e+01


### 4 - Tratamento de dados ausentes com o uso de arquivo _shapefile_

In [9]:
df.isna().any(axis=0)

ano                        False
mes                        False
data_hora                  False
bioma                       True
sigla_uf                   False
sigla_uf_nome              False
id_municipio               False
id_municipio_nome          False
latitude                   False
longitude                  False
satelite                    True
dias_sem_chuva              True
precipitacao                True
risco_fogo                  True
potencia_radiativa_fogo     True
dtype: bool

In [10]:
df[['bioma']].isna().any(axis=1).sum()

8

In [11]:
df['bioma'].unique()

array(['Cerrado', 'Mata Atlântica', 'Caatinga', 'Amazônia', 'Pantanal',
       'Pampa', None], dtype=object)

In [12]:
df_dados_ausentes = df[df[['bioma']].isna().any(axis=1)]
df_dados_ausentes.head(5)

,ano,mes,data_hora,bioma,sigla_uf,sigla_uf_nome,id_municipio,id_municipio_nome,latitude,longitude,satelite,dias_sem_chuva,precipitacao,risco_fogo,potencia_radiativa_fogo
1161720,2023,2,2023-02-08 04:14:00,None,RS,Rio Grande do Sul,3137304,Lagoa dos Patos,-32.11578,-52.14998,NOAA-20,0.0,25.32,0.0,0.8
2814785,2024,1,2024-01-30 01:08:35,None,RS,Rio Grande do Sul,3137304,Lagoa dos Patos,-30.02220,-51.30270,METOP-C,0.0,0.00,0.0,NaN
2916890,2023,2,2023-02-15 17:45:00,None,RS,Rio Grande do Sul,3137304,Lagoa dos Patos,-31.97725,-52.11386,NOAA-20,0.0,3.79,0.0,4.1
4320918,2023,8,2023-08-30 17:24:00,None,PR,Paraná,4120358,Pranchita,-25.98349,-53.81910,NPP-375,12.0,0.00,0.6,8.9
8177948,2023,2,2023-02-18 19:27:56,None,RS,Rio Grande do Sul,3137304,Lagoa dos Patos,-32.03790,-52.19320,GOES-16,0.0,0.00,0.0,109.7


In [13]:
# Caminho para o arquivo .shp
caminho_shp = 'Biomas_250mil/lm_bioma_250.shp'

# Carrega o shapefile
gdf = gpd.read_file(caminho_shp)

# Cria o mapa centrado no centroide da união das geometrias
centro = gdf.geometry.unary_union.centroid.coords[:][0][::-1]  # (lat, lon)
mapa = folium.Map(location=centro, zoom_start=4)

# Adiciona o shapefile como camada GeoJSON com tooltip
campos_tooltip = gdf.columns.drop('geometry').tolist()

# Gera uma cor aleatória para cada feição
gdf['cor'] = ['#%06x' % random.randint(0, 0xFFFFFF) for _ in range(len(gdf))]

folium.GeoJson(
    gdf,
    style_function=lambda feature: {
        'fillColor': feature['properties']['cor'],
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.3
    },
    tooltip=GeoJsonTooltip(
        fields=campos_tooltip,
        aliases=campos_tooltip,
        localize=True,
        sticky=True
    )
).add_to(mapa)

# Converte df_dados_ausentes para GeoDataFrame de pontos
gdf_pontos = gpd.GeoDataFrame(
    df_dados_ausentes.copy(),
    geometry=gpd.points_from_xy(df_dados_ausentes['longitude'], df_dados_ausentes['latitude']),
    crs="EPSG:4326"  # coordenadas em WGS84
)

# Adiciona os pontos ao mapa
# Adiciona os pontos do GeoDataFrame gdf_pontos ao mapa
for ponto in gdf_pontos.geometry:
    folium.CircleMarker(
        location=[ponto.y, ponto.x],  # ponto.y é a latitude, ponto.x é a longitude
        radius=3,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        popup=f"Lat: {ponto.y}, Lon: {ponto.x}"
    ).add_to(mapa)

# Exibe o mapa
#mapa
#comentado para reduzir o tamanho do arquivo

In [14]:
valores_gdf = np.sort(gdf['Bioma'].dropna().unique())
valores_df = np.sort(df['bioma'].dropna().unique())
valores_gdf == valores_df

array([ True,  True,  True,  True,  True,  True])

In [15]:
# 1. Converte df_dados_ausentes para GeoDataFrame de pontos
gdf_pontos = gpd.GeoDataFrame(
    df_dados_ausentes.copy(),
    geometry=gpd.points_from_xy(df_dados_ausentes['longitude'], df_dados_ausentes['latitude']),
    crs="EPSG:4326"  # WGS84
)

# 2. Reprojetar o gdf para o mesmo CRS se necessário
if gdf.crs != gdf_pontos.crs:
    gdf = gdf.to_crs(gdf_pontos.crs)

# 3. Faz o join espacial para ver onde há sobreposição
sobreposicoes = gpd.sjoin(gdf_pontos, gdf, predicate="intersects", how="inner")

sobreposicoes['Bioma']

1161720              Pampa
2814785              Pampa
2916890              Pampa
4320918     Mata Atlântica
8177948              Pampa
10933230    Mata Atlântica
15197482             Pampa
16879620    Mata Atlântica
Name: Bioma, dtype: object

In [16]:
df.loc[df.index.isin(sobreposicoes.index), 'bioma'] = sobreposicoes['Bioma'].values

In [17]:
df[['bioma']].isna().any(axis=1).sum()

0

### 5 - Criando novas colunas

In [18]:
df['estacao'] = df['mes'].apply(estacao_ano)
df['classificacao_risco'] = df['risco_fogo'].apply(classificar_risco)
df['mes_abrev'] = df['mes'].apply(lambda x: calendar.month_abbr[x].capitalize())

In [19]:
df.head(10)

,ano,mes,data_hora,bioma,sigla_uf,sigla_uf_nome,id_municipio,id_municipio_nome,latitude,longitude,satelite,dias_sem_chuva,precipitacao,risco_fogo,potencia_radiativa_fogo,estacao,classificacao_risco,mes_abrev
0,2024,7,2024-07-27 12:56:20,Cerrado,BA,Bahia,2909307,Correntina,-13.16580,-45.22000,GOES-16,107.0,0.0,1.00,125.8,Inverno,Crítico,Jul
1,2024,7,2024-07-27 15:26:20,Cerrado,BA,Bahia,2909307,Correntina,-13.16580,-45.22000,GOES-16,107.0,0.0,1.00,77.1,Inverno,Crítico,Jul
2,2024,7,2024-07-27 00:16:20,Cerrado,BA,Bahia,2909307,Correntina,-13.16580,-45.22000,GOES-16,107.0,0.0,1.00,206.2,Inverno,Crítico,Jul
3,2024,7,2024-07-27 00:56:20,Cerrado,BA,Bahia,2909307,Correntina,-13.18510,-45.21680,GOES-16,107.0,0.0,1.00,163.7,Inverno,Crítico,Jul
4,2024,7,2024-07-29 17:28:00,Cerrado,BA,Bahia,2917334,Iuiú,-14.43488,-43.43603,AQUA_M-T,107.0,0.0,1.00,350.8,Inverno,Crítico,Jul
5,2024,7,2024-07-28 16:18:00,Cerrado,BA,Bahia,2928901,São Desidério,-13.03870,-44.73677,NOAA-21,107.0,0.0,1.00,11.3,Inverno,Crítico,Jul
6,2024,7,2024-07-25 17:11:00,Mata Atlântica,MG,Minas Gerais,3102209,Alvarenga,-19.38934,-41.60167,NOAA-21,107.0,0.0,1.00,9.5,Inverno,Crítico,Jul
7,2024,7,2024-07-19 16:09:00,Mata Atlântica,MG,Minas Gerais,3116803,Coluna,-18.31096,-42.83261,NPP-375,107.0,0.0,0.97,6.4,Inverno,Crítico,Jul
8,2024,7,2024-07-22 04:21:00,Cerrado,MG,Minas Gerais,3126703,Francisco Sá,-16.52762,-43.55104,NPP-375D,107.0,0.0,1.00,3.4,Inverno,Crítico,Jul
9,2024,7,2024-07-23 04:04:00,Cerrado,MG,Minas Gerais,3147402,Paraopeba,-19.39766,-44.29760,NPP-375D,107.0,0.0,1.00,0.9,Inverno,Crítico,Jul


### 5 - Verificando a existência de duplicidades

In [20]:
df.duplicated().sum()

0

### 6 - Salvando os dados 

In [21]:
df.to_parquet('dados_tratados.parquet')
df.to_csv('dados_tratados.csv', index=False)

### Dados sobre versão

In [22]:
import sys
import pandas as pd
import numpy as np
import matplotlib
import basedosdados as bd
import geopandas as gpd
import folium
import shapely
print("Python:", sys.version)
print("pandas:", pd.__version__)
print("numpy:", np.__version__)
print("matplotlib:", matplotlib.__version__)
print("basedosdados:", bd.__version__)
print("geopandas:", gpd.__version__)
print("folium:", folium.__version__)
print("shapely:", shapely.__version__)

Python: 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
pandas: 2.2.3
numpy: 1.23.5
matplotlib: 3.9.4
basedosdados: 2.0.2
geopandas: 1.0.1
folium: 0.19.5
shapely: 2.0.7
